In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["NCCL_IB_DISABLE"] = "1"
os.environ["NCCL_P2P_DISABLE"] = "1"

In [2]:
import torch
from datasets import load_dataset, load_metric,concatenate_datasets
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments,AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from torch.utils.data import DataLoader
from tqdm import tqdm
import evaluate
import numpy as np
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score
from datasets import Dataset, DatasetDict, load_dataset, load_metric



In [3]:
from datasets import Dataset, DatasetDict, load_dataset, load_metric
from transformers import T5Tokenizer
from tqdm import tqdm
import json
import os

# Load dataset (example using CoLA)
dataset = load_dataset("glue", "cola")

# Tokenizer and metrics
tokenizer = T5Tokenizer.from_pretrained("t5-base")
metric = load_metric("glue", "cola")

# Prepare dataset
def preprocess_function(examples):
    input_ids = []
    attention_mask = []
    labels = []
    for i in tqdm(range(len(examples))):
        tokenized = tokenizer(examples["sentence"][i], truncation=True, max_length=512, padding="max_length")
        input_ids.append(tokenized["input_ids"])
        attention_mask.append(tokenized["attention_mask"])
        labels.append(examples["label"][i])
    return {"input_ids": input_ids, "attention_mask": attention_mask, "label": labels}

# Convert to Dataset object and preprocess
train_dataset = Dataset.from_dict(preprocess_function(dataset['train']))
eval_dataset = Dataset.from_dict(preprocess_function(dataset["validation"]))
test_dataset = Dataset.from_dict(preprocess_function(dataset["test"]))



You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/tmp/ipykernel_108834/1636946568.py:12: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "cola")
/media/.venv/lib/python3.11/site-packages/datasets/load.py:759: FutureWarning: The repository for glue contains custom code which must be exec

In [4]:
len(train_dataset)

8551

In [5]:
num_epochs = 15
concatenated_data = concatenate_datasets([train_dataset, train_dataset])
for i in tqdm(range(1,num_epochs-1)):
    concatenated_data = concatenate_datasets([concatenated_data, train_dataset])
train_dataset = concatenated_data.shuffle(seed=42)  
print(f"Number of rows {len(train_dataset)}")

100%|█████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 1165.66it/s]

Number of rows 128265


In [6]:
set_batch_size = 8
num_rows = len(train_dataset)

# Round down to the nearest hundred
rounded_num_rows = (num_rows // set_batch_size) * set_batch_size

# Truncate the dataset
train_dataset = train_dataset.select(range(rounded_num_rows))

In [7]:
# Create a DatasetDict
dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": eval_dataset,
    "test": test_dataset
})

# Save the dataset to disk
save_path = "./preprocessed_cola_dataset"
dataset_dict.save_to_disk(save_path)

# Save preprocessing args to a json file
preprocessing_args = {
    "sequence_length": 512,
    "tokenizer": "t5-base"
}

with open(os.path.join(save_path, "args.json"), "w") as f:
    json.dump(preprocessing_args, f)

print(f"Dataset and preprocessing arguments saved to {save_path}")


Saving the dataset (0/1 shards):   0%|          | 0/128264 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1043 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1063 [00:00<?, ? examples/s]

Dataset and preprocessing arguments saved to ./preprocessed_cola_dataset


In [8]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [9]:
#keep origin weight = false !!!!!!!!!!!
print(f"Number of rows {len(train_dataset)}")

Number of rows 128264


In [2]:
!torchrun --nproc_per_node=1 torchrun_main.py \
    --model_name_or_path t5-base \
    --dataset_path ./preprocessed_cola_dataset \
    --batch_size 16 \
    --lr 3e-4 \
    --max_length 512 \
    --use_peft True \
    --cycle_length 32500 \
    --restart_warmup_steps 200 \
    --scheduler cosine_restarts \
    --warmup_steps 200 \
    --reset_optimizer_on_relora True \
    --num_training_steps 130000 \
    --save_every 130000 \
    --eval_every 5200 \
    --tags relora_t5_base \
    --use_double_quant False \
    --workers 4 \
    --seed 42\
    --relora 32500\
    --lora_r 64\
    --optimizer_magnitude_pruning 0.1\
    

          




2024-07-29 21:02:04.141 | INFO     | __main__:main:356 - Global rank 0, local rank 0, device: 0
2024-07-29 21:02:04.142 | INFO     | __main__:main:360 - Process group initialized
wandb: Currently logged in as: delyanhristov06 (delyanhristov06-). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.17.5
wandb: Run data is saved locally in /home/delyanh/Projects/ReLoRa/wandb/run-20240729_210205-l1fw4q4h
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run dashing-river-78
wandb: ⭐️ View project at https://wandb.ai/delyanhristov06-/peft_pretraining
wandb: 🚀 View run at https://wandb.ai/delyanhristov06-/peft_pretraining/runs/l1fw4q4h
2024-07-29 21:02:06.354 | INFO     | __main__:main:430 - Using dist with rank 0 (only rank 0 will log)
2024-07-29 21:02:06.354 | INFO     | __main__:main:431 - ****************************************
2024-07-29 21:02:06.354 | INFO     | __main__:main:432 - Starting training with the arguments
2024-07-29 21:02:06.

In [11]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.
